# Análisis Descriptivo: Mapping Review IA y ML en Educación Matemática K-12

**MQ1: ¿Cuántos estudios se han publicado a lo largo de los años?**

Este notebook realiza un análisis descriptivo completo del corpus de publicaciones sobre IA y ML en educación matemática K-12.

## 1. Configuración del Entorno

In [ ]:
# Instalación de dependencias
!pip install pandas numpy matplotlib seaborn plotly wordcloud nltk scikit-learn textblob

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos desde GitHub

In [ ]:
# Cargar el dataset desde GitHub
# IMPORTANTE: Cambiar la URL por tu repositorio real
url = "https://raw.githubusercontent.com/mlproyecto/doctorado/main/MappingReview.csv"
df = pd.read_csv(url, sep=';', encoding='latin-1')

print(f"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas")
print("\nPrimeras 5 filas:")
df.head()

In [ ]:
# Información básica del dataset
print("Información del dataset:")
print(df.info())

print("\nEstadísticas descriptivas:")
print(df.describe())

print("\nValores únicos por columna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

## 3. Limpieza y Preparación de Datos

In [ ]:
# Limpieza de datos
# Convertir Year a numérico
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Limpiar columnas de presencia en bases de datos
presence_columns = ['Present in Eric', 'Present in IEEE', 'Present in Scopus', 'Present in WoS']
for col in presence_columns:
    df[col] = df[col].fillna('No')
    df[col] = df[col].str.strip()

print("Datos limpiados exitosamente")
print(f"Rango de años: {df['Year'].min()} - {df['Year'].max()}")

## 4. Análisis Temporal (MQ1)

In [ ]:
# Análisis de publicaciones por año
yearly_publications = df['Year'].value_counts().sort_index()

# Gráfico de líneas temporal
plt.figure(figsize=(15, 8))
plt.plot(yearly_publications.index, yearly_publications.values, marker='o', linewidth=3, markersize=8)
plt.fill_between(yearly_publications.index, yearly_publications.values, alpha=0.3)
plt.title('Evolución de Publicaciones sobre IA y ML en Educación Matemática K-12 (2020-2025)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Año', fontsize=14)
plt.ylabel('Número de Publicaciones', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(yearly_publications.index, rotation=45)

# Agregar valores en los puntos
for i, v in enumerate(yearly_publications.values):
    plt.text(yearly_publications.index[i], v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total de publicaciones: {len(df)}")
print(f"Año con más publicaciones: {yearly_publications.idxmax()} ({yearly_publications.max()} publicaciones)")
print(f"Año con menos publicaciones: {yearly_publications.idxmin()} ({yearly_publications.min()} publicaciones)")

In [ ]:
# Gráfico interactivo con Plotly
fig = px.line(x=yearly_publications.index, y=yearly_publications.values,
              title='Evolución Temporal de Publicaciones',
              labels={'x': 'Año', 'y': 'Número de Publicaciones'})

fig.update_traces(mode='lines+markers', line=dict(width=3), marker=dict(size=8))
fig.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    hovermode='x unified'
)

fig.show()

## 5. Estadísticas Descriptivas Generales

In [ ]:
# Estadísticas generales
print("=== ESTADÍSTICAS DESCRIPTIVAS GENERALES ===\n")

print(f"Total de publicaciones analizadas: {len(df)}")
print(f"Período de análisis: {df['Year'].min()} - {df['Year'].max()}")
print(f"Promedio de publicaciones por año: {len(df) / (df['Year'].max() - df['Year'].min() + 1):.1f}")

# Análisis por tipo de publicación
print("\n=== DISTRIBUCIÓN POR TIPO DE PUBLICACIÓN ===")
type_dist = df['Type of Publication'].value_counts()
print(type_dist)

# Gráfico de tipos de publicación
plt.figure(figsize=(12, 8))
colors = plt.cm.Set3(np.linspace(0, 1, len(type_dist)))
plt.pie(type_dist.values, labels=type_dist.index, autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('Distribución por Tipo de Publicación', fontsize=16, fontweight='bold')
plt.axis('equal')
plt.show()

In [ ]:
# Análisis de presencia en bases de datos
print("\n=== PRESENCIA EN BASES DE DATOS ===")
presence_summary = df[presence_columns].apply(lambda x: x.value_counts())
print(presence_summary)

# Gráfico de barras para presencia en bases de datos
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, col in enumerate(presence_columns):
    counts = df[col].value_counts()
    axes[i].bar(counts.index, counts.values, color=['#2E8B57', '#FF6B6B'])
    axes[i].set_title(f'Presencia en {col.split("Present in ")[1]}', fontweight='bold')
    axes[i].set_ylabel('Número de Publicaciones')
    
    # Agregar valores en las barras
    for j, v in enumerate(counts.values):
        axes[i].text(j, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Análisis de Tendencias

In [ ]:
# Análisis de tendencias por año y tipo de publicación
trend_analysis = df.groupby(['Year', 'Type of Publication']).size().unstack(fill_value=0)

plt.figure(figsize=(15, 8))
trend_analysis.plot(kind='bar', stacked=True, figsize=(15, 8))
plt.title('Tendencias de Publicación por Año y Tipo', fontsize=16, fontweight='bold')
plt.xlabel('Año', fontsize=14)
plt.ylabel('Número de Publicaciones', fontsize=14)
plt.legend(title='Tipo de Publicación', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nTendencias por año y tipo de publicación:")
print(trend_analysis)

## 7. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo
print("=== RESUMEN EJECUTIVO ===\n")

print(f"📊 Total de publicaciones analizadas: {len(df)}")
print(f"📅 Período de análisis: {df['Year'].min()} - {df['Year'].max()}")
print(f"📈 Año con mayor actividad: {yearly_publications.idxmax()} ({yearly_publications.max()} publicaciones)")
print(f"📉 Año con menor actividad: {yearly_publications.idxmin()} ({yearly_publications.min()} publicaciones)")
print(f"📊 Promedio anual: {yearly_publications.mean():.1f} publicaciones")

# Tipo más común
most_common_type = type_dist.index[0]
print(f"📝 Tipo de publicación más común: {most_common_type} ({type_dist.iloc[0]} publicaciones)")

# Base de datos más utilizada
most_used_db = presence_summary.loc['Yes'].idxmax()
print(f"🔍 Base de datos más utilizada: {most_used_db.split('Present in ')[1]}")

print("\n=== CONCLUSIONES ===")
print("1. El campo de IA y ML en educación matemática K-12 está en crecimiento")
print("2. La mayoría de publicaciones son artículos de investigación")
print("3. Hay una tendencia creciente en los últimos años")
print("4. Se utilizan múltiples bases de datos para la búsqueda")